In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

The incompressible 2D Navier-Stokes equations are:

\begin{align}
    \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} &= -\frac{1}{\rho} \frac{\partial p}{\partial x} + \nu \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right), \\
    \frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} &= -\frac{1}{\rho} \frac{\partial p}{\partial y} + \nu \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right), \\
    \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} &= 0,
\end{align}

where $u$ and $v$ are the velocity components in the $x$ and $y$ directions, respectively, $p$ is the pressure, $\rho$ is the fluid density, and $\nu$ is the kinematic viscosity.

The pressure Poisson equation is:

\begin{equation}
    \nabla^2 p = b,
\end{equation}

where $p$ is the pressure, $\nabla^2$ is the Laplace operator, and $b$ is the source term.

In 2D Cartesian coordinates, the pressure Poisson equation can be written as:

\begin{equation}
    \frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2} = b.
\end{equation}

The source term $b$ in the pressure Poisson equation is computed as:

\begin{equation}
    b = \rho \left( \frac{1}{\Delta t} \left( \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} \right) - \left(\frac{\partial u}{\partial x}\right)^2 - 2 \frac{\partial u}{\partial y} \frac{\partial v}{\partial x} - \left(\frac{\partial v}{\partial y}\right)^2 \right),
\end{equation}

where $b$ is the source term, $\rho$ is the fluid density, $u$ and $v$ are the velocity components in the $x$ and $y$ directions, respectively, $\Delta t$ is the time step size, and $\frac{\partial}{\partial x}$ and $\frac{\partial}{\partial y}$ represent the partial derivatives with respect to $x$ and $y$, respectively.


In [ ]:
# Initial conditions
# u0 = 1 
# v0 = 0

# u, v = 0, throughout the domain
# p = 101325 Pa = 1 atm, throughout the domain

# Boundary conditions

# u0_left = 1
# u_left = 1
# v_left = 0
# TODO (box)

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_units):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        in_units = input_dim
        for units in hidden_units:
            self.layers.append(nn.Linear(in_units, units))
            in_units = units
        self.layers.append(nn.Linear(in_units, output_dim))

    def forward(self, input):
        for layer in self.layers[:-1]:
            output = torch.tanh(layer(input))
            input = output
        output = self.layers[-1](input)
        return output

    def loss(self, x_f, y_f, t_f, u0, v0, p0, mu, rho, dt):
        # TODO fix initial and boundary condtions

        output = self(torch.cat([x_f, y_f, t_f], dim=1))
        u = output[:, 0]
        v = output[:, 1]
        p = output[:, 2]

        u_t = torch.autograd.grad(u, t_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_x = torch.autograd.grad(u, x_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_y = torch.autograd.grad(u, y_f, grad_outputs=torch.ones_like(u), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_xx = torch.autograd.grad(u_x, x_f, grad_outputs=torch.ones_like(u_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        u_yy = torch.autograd.grad(u_y, y_f, grad_outputs=torch.ones_like(u_y), create_graph=True, retain_graph=True, only_inputs=True)[0]

        v_t = torch.autograd.grad(v, t_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_x = torch.autograd.grad(v, x_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_y = torch.autograd.grad(v, y_f, grad_outputs=torch.ones_like(v), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_xx = torch.autograd.grad(v_x, x_f, grad_outputs=torch.ones_like(v_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        v_yy = torch.autograd.grad(v_y, y_f, grad_outputs=torch.ones_like(v_y), create_graph=True, retain_graph=True, only_inputs=True)[0]

        p_x = torch.autograd.grad(p, x_f, grad_outputs=torch.ones_like(p), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_xx = torch.autograd.grad(p_x, x_f, grad_outputs=torch.ones_like(p_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_y = torch.autograd.grad(p, y_f, grad_outputs=torch.ones_like(p), create_graph=True, retain_graph=True, only_inputs=True)[0]
        p_yy = torch.autograd.grad(p_y, y_f, grad_outputs=torch.ones_like(p_y), create_graph=True, retain_graph=True, only_inputs=True)[0]

        b = rho * ( 1/dt * (u_x + v_y) - u_x**2 - 2*u_y*v_x - v_y**2)

        f1 = u_t + u*u_x + v*u_y + (1/rho) * p_x - mu * (u_xx + u_yy)
        f2 = v_t + u*v_x + v*v_y + (1/rho) * p_y - mu * (v_xx + v_yy)
        f3 = u_x + v_y
        f4 = p_xx + p_yy - b

        # Initial condition loss
        output_init = self(torch.cat([x_f, y_f, torch.zeros_like(t_f)], dim=1))
        u0_pred = output_init[:, 0]
        v0_pred = output_init[:, 1]
        p0_pred = output_init[:, 2]
        
        u0_true = 1 # TODO dims
        v0_true = 0 # TODO dims
        p0_true = 0 # TODO dims
        
        ic_loss_u = torch.mean(torch.square(u0_pred - u0_true))
        ic_loss_v = torch.mean(torch.square(v0_pred - v0_true))
        ic_loss_p = torch.mean(torch.square(p0_pred - p0_true))

        # Boundary conditions loss
        # TODO

        # Combine PDE residual, initial condition, and boundary condition losses
        total_loss = torch.mean(torch.square(f1)) + \
                    torch.mean(torch.square(f2)) + \
                    torch.mean(torch.square(f3)) + \
                    torch.mean(torch.square(f4)) + \
                    ic_loss_u + ic_loss_v + ic_loss_p 

        return total_loss


In [ ]:
x_max = 1
y_max = 1
t_max = 1

Nx = 100
Ny = 100
Nt = 100

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
t_test = np.linspace(0, t_max, Nt)
x_grid, y_grid, t_grid = np.meshgrid(x_test, y_test, t_test)

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

input_dim = 3
output_dim = 3
hidden_units = [64, 64, 64, 64]
pinn = PINN(input_dim, output_dim, hidden_units).to(device)

optimizer = torch.optim.Adam(pinn.parameters(), lr=0.01)
epochs = 1000
Nf = 1000

# Density (rho): 1.184 kg/m³
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1184
mu = 1.81e-5

for epoch in range(epochs):
    x_f = torch.tensor(np.random.uniform(0, x_max, size=(Nf, 1)), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    y_f = torch.tensor(np.random.uniform(0, y_max, size=(Nf, 1)), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    t_f = torch.tensor(np.random.uniform(0, t_max, size=(Nf, 1)), dtype=torch.float32, device=device, requires_grad=True) # collocation points
    
    optimizer.zero_grad()
    loss = pinn.loss(x_f, y_f, t_f, 0, 0, 0, mu, rho, dt)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

# Evaluate the model at the grid points
# x_test_tensor = torch.tensor(x_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
# t_test_tensor = torch.tensor(t_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
# T_test = pinn(torch.cat([x_test_tensor, t_test_tensor], dim=1))
# T_test = T_test.detach().cpu().numpy().reshape(Nt, Nx) # maybe reversed

# # Create a contour plot of the temperature distribution
# plt.figure()
# plt.contourf(x_test, t_test, T_test, cmap="jet")
# plt.xlabel("x")
# plt.ylabel("t")
# plt.title("Temperature Distribution (PINN)")
# plt.colorbar(label="Temperature")
# plt.show()